In [ ]:
from deep_translator import GoogleTranslator
import spacy
import services.influencers_sevice as influencers_service
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords

In [16]:
translator = GoogleTranslator(source='auto', target='en')
influencers = influencers_service.get_influencers()
posts=influencers_service.get_posts()
stopwords_en = stopwords.words('english')

In [17]:
def translate_post(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        try:
            translated_content = translator.translate(content)
            print("after tanslation", translated_content)
            influencers_service.update_post(post, key,translated_content)
    
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this caption.")

In [18]:
def translate_hashtags(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        translated_hashtags = []
        for hashtag in hashtags:
            try:
                translated_hashtag = translator.translate(hashtag)
                print(translated_hashtag)
                translated_hashtags.append(translated_hashtag)
                post['hashtags'] = translated_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [ ]:
def lemmatize_text(text):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    allowed_postags = ["NOUN", "AD", "VERB", "ADV"]
    doc = nlp(text)
    new_text = []
    for word in doc:
        new_text.append(word.lemma_)
    lemmatized_text = " ".join(new_text)
    return lemmatized_text

In [ ]:
def lemmatize_post(posts,key):
    for post in posts: 
        try:
            content = post.get(key)
            lemmatized_content = lemmatize_text(content)
            influencers_service.update_post(post, key , lemmatized_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this hashtag.")

In [ ]:
def lemmatize_hashtags(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        lemmatized_hashtags = []
        for hashtag in hashtags:
            try:
                lemmatized_hashtag = lemmatize_text(hashtag)
                lemmatized_hashtags.append(lemmatized_hashtag )
                post['hashtags'] = lemmatized_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [ ]:
def remove_emojis(text):
    return unidecode(text)


In [ ]:
def remove_hashtags_emojis(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        hashtags = []
        for hashtag in hashtags:
            try:
                updated_hashtags = remove_emojis(hashtag)
                updated_hashtags .append(updated_hashtags)
                post['hashtags'] = updated_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [ ]:
def remove_post_emojis(posts,key):
    for post in posts:
        content = post.get(key)
        no_emoji_content = remove_emojis(content)
        influencers_service.update_post(post, key, no_emoji_content)

In [ ]:
def remove_stopwords(text, stopwords_list):
    words = text.split()
    words = [word for word in words if word not in stopwords_list]
    return ' '.join(words)

In [ ]:
def remove_post_stopwords(posts,key):
    for post in posts:
        content = post.get(key)
        try:
            no_stopwords_content = remove_stopwords(content, stopwords_en)
            influencers_service.update_post(post, key, no_stopwords_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this post")

In [ ]:
def remove_hashtags_stop_words(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        hashtags = []
        for hashtag in hashtags:
            try:
                updated_hashtags = remove_stopwords(hashtag)
                updated_hashtags .append(updated_hashtags)
                post['hashtags'] = updated_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [ ]:
#translate_post(posts,'firstComment')
translate_post(posts, 'caption')
translate_hashtags(posts)

In [ ]:
lemmatize_post(posts,'firstComment')
lemmatize_post(posts, 'caption')
lemmatize_hashtags(posts)        

In [ ]:
remove_post_emojis(posts,'firstcomment')
remove_post_emojis(posts,'caption')
remove_hashtags_emojis(posts)

In [ ]:
remove_post_stopwords(posts,'firstcomment')
remove_post_stopwords(posts,'caption')
remove_hashtags_stop_words(posts)